Есеп 1. Анықтамалар

Ансамбль әдісі: бірнеше модельдің болжамдарын біріктіріп (voting/averaging/weighting/stacking), бір ортақ шешім шығаратын тәсіл.

Неге бірнеше модель бір модельден жақсы жұмыс істеуі мүмкін: қателерін «орташа» қылады; модельдер әртүрлі қате жіберсе, біріктіру variance-ті азайтып, жалпылауды күшейтеді.

Bias / Variance:

Bias: модельдің шамадан тыс қарапайымдылығынан туатын жүйелі қате (underfitting).

Variance: дерекке тым қатты тәуелділік, оқыту жиынтығындағы өзгеріске сезімталдық (overfitting).

Есеп 2. Bagging идеясы

Bootstrap sample: бастапқы train деректерінен қайта таңдаумен (replacement) алынған сол көлемдегі үлгі.

Bagging қандай проблеманы азайтады: негізінен variance (overfitting тәуекелі).

Bagging параллель орындала ма? Неге?: иә, әр модель тәуелсіз bootstrap-та оқытылады, бір-біріне тәуелді емес.

Есеп 3. Random Forest ерекшелігі

Bootstrap sampling — тән

Feature randomness — тән

Sequential learning — тән емес

Decision trees — тән

Gradient optimization — тән емес

🟡 2-деңгей. Есептеу және логика
Есеп 4. Majority voting

Берілген: M1=1, M2=0, M3=1

Ансамбль болжамы: 1 (көпшілік 1).

Бір модель салмағы 2 есе үлкен болса өзгере ме?

Егер салмағы 2 есе модель M2 болса: салмақтар (M1=1, M2=2, M3=1) → 0 дауысы=2, 1 дауысы=2 → тең түседі (tie). Tie-breaking ережесіне байланысты (әдетте алдын ала келісім керек).

Егер салмағы 2 есе модель M1 немесе M3 болса: 1 дауысы басым болып қалады → нәтиже 1.

Есеп 6. Bagging (Python) — Decision Tree base, n_estimators=30, accuracy, бір ағашпен салыстыру

In [2]:
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

# 1) Dataset
data = load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 2) Single Decision Tree
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
pred_dt = dt.predict(X_test)
acc_dt = accuracy_score(y_test, pred_dt)

# 3) Bagging with Decision Tree base model
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=42),
    n_estimators=30,
    random_state=42,
    n_jobs=-1
)
bag.fit(X_train, y_train)
pred_bag = bag.predict(X_test)
acc_bag = accuracy_score(y_test, pred_bag)

print(f"Single Decision Tree accuracy: {acc_dt:.4f}")
print(f"Bagging (30 trees) accuracy:   {acc_bag:.4f}")
print(f"Accuracy difference (Bag - DT): {acc_bag - acc_dt:+.4f}")

Single Decision Tree accuracy: 0.9123
Bagging (30 trees) accuracy:   0.9386
Accuracy difference (Bag - DT): +0.0263


Есеп 7. Random Forest гиперпараметрлері

n_estimators артса не болады?: әдетте variance азаяды, accuracy тұрақтанады; уақыт/жады шығыны өседі; белгілі бір межеден кейін ұтыс азаяды.

max_depth азайса әсері: ағаштар қарапайымдайды → bias өседі, variance азаяды; overfitting төмендейді, бірақ underfitting туындауы мүмкін.

Неліктен Random Forest overfitting-ке төзімді?: bootstrap + feature randomness ағаштарды әртүрлі қылады; орташа/дауыс беру variance-ті түсіреді.

🔵 4-деңгей. Gradient Boosting
Есеп 8. Бустинг логикасы

Неліктен Gradient Boosting sequential?: әр жаңа модель алдыңғылардың қателерін түзету үшін (residual/градиент бағытымен) үйренеді, сондықтан тәуелді қадамдар тізбегі пайда болады.

Residuals (қалдықтар): алдыңғы модельдің қателігі; бинар/көпкласста — loss функциясының теріс градиенті түрінде қолданылады.

Learning rate рөлі: әр қадамның үлесін «бәсеңдетеді»; кішірек rate → тұрақтырақ, бірақ көбірек estimator керек; тым үлкен rate → overfitting/тұрақсыздық қаупі.

Есеп 9. Gradient Boosting (Python) — n_estimators=50,100,200 салыстыру

In [3]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

n_list = [50, 100, 200]
results = {}

for n in n_list:
    gb = GradientBoostingClassifier(n_estimators=n, random_state=42)
    gb.fit(X_train, y_train)
    pred = gb.predict(X_test)
    acc = accuracy_score(y_test, pred)
    results[n] = acc
    print(f"GradientBoosting n_estimators={n:3d} -> accuracy={acc:.4f}")

best_n = max(results, key=results.get)
print(f"\nBest n_estimators by test accuracy: {best_n} (acc={results[best_n]:.4f})")

GradientBoosting n_estimators= 50 -> accuracy=0.9474
GradientBoosting n_estimators=100 -> accuracy=0.9561
GradientBoosting n_estimators=200 -> accuracy=0.9561

Best n_estimators by test accuracy: 100 (acc=0.9561)


🔴 5-деңгей. Stacking
Есеп 10. Stacking құрылымы

Level-0 модельдер: Decision Tree, Random Forest, SVM (бұлар базалық модельдер).

Мета-модель: Logistic Regression (жиі), немесе Linear model, Gradient Boosting, т.б.

Неліктен train деректерді тікелей қолдануға болмайды?: meta-model базалық модельдердің train-дегі «тым жақсы» болжамына үйреніп, leakage арқылы overfitting күшейеді; дұрысы — out-of-fold болжам.

Есеп 11. Stacking (Python) — ≥2 base model, Logistic Regression meta, accuracy, RF-пен салыстыру

In [3]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

# 1) Dataset (moved from previous cell to ensure definition)
data = load_breast_cancer()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Base models
estimators = [
    ("dt", DecisionTreeClassifier(random_state=42)),
    ("svm", SVC(probability=True, random_state=42))
]

# Meta model
meta = LogisticRegression(max_iter=5000, random_state=42)

stack = StackingClassifier(
    estimators=estimators,
    final_estimator=meta,
    cv=5,
    n_jobs=-1,
    passthrough=False
)

stack.fit(X_train, y_train)
pred_stack = stack.predict(X_test)
acc_stack = accuracy_score(y_test, pred_stack)

rf = RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)
acc_rf = accuracy_score(y_test, pred_rf)

print(f"Stacking accuracy:      {acc_stack:.4f}")
print(f"RandomForest accuracy:  {acc_rf:.4f}")
print(f"Difference (Stack - RF): {acc_stack - acc_rf:+.4f}")

Stacking accuracy:      0.9211
RandomForest accuracy:  0.9561
Difference (Stack - RF): -0.0351


🟣 6-деңгей. Аналитикалық
Есеп 12. Bias–Variance салыстыру

Single Decision Tree: bias төмен/орташа, variance жоғары.

Bagging: bias шамамен сол күйі, variance төмен.

Random Forest: bias сәл өседі/ұқсас, variance bagging-тен де төмен (feature randomness әсері).

Gradient Boosting: bias төмендейді (күшті fit), variance өсіп кетуі мүмкін (әсіресе үлкен depth/көп estimator/үлкен learning rate кезінде).

Есеп 13. Қай әдісті таңдау керек?

Шағын датасет: көбіне Random Forest (тұрақты), немесе Bagging (қарапайым).

Үлкен noisy деректер: Random Forest немесе Bagging (variance-ті басады).

Feature саны өте көп: Random Forest (feature subsampling пайдалы), кейде linear + regularization (егер өте sparse/жоғары өлшем).

Ең жоғары accuracy қажет: жиі Gradient Boosting немесе Stacking (дұрыс баптаумен).

Қорытынды есеп (Mini-project) — бір датасетте 4 әдіс салыстыру

In [4]:
import time
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def fit_eval(model, name):
    t0 = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - t0
    pred = model.predict(X_test)
    acc = accuracy_score(y_test, pred)
    return name, acc, train_time

models = []

# Bagging
models.append((
    "Bagging",
    BaggingClassifier(
        estimator=DecisionTreeClassifier(random_state=42),
        n_estimators=200,
        random_state=42,
        n_jobs=-1
    )
))

# Random Forest
models.append((
    "RandomForest",
    RandomForestClassifier(
        n_estimators=500,
        random_state=42,
        n_jobs=-1
    )
))

# Gradient Boosting
models.append((
    "GradientBoosting",
    GradientBoostingClassifier(
        n_estimators=200,
        random_state=42
    )
))

# Stacking
stack = StackingClassifier(
    estimators=[
        ("rf", RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1)),
        ("svm", SVC(probability=True, random_state=42))
    ],
    final_estimator=LogisticRegression(max_iter=5000, random_state=42),
    cv=5,
    n_jobs=-1
)
models.append(("Stacking", stack))

results = []
for name, model in models:
    results.append(fit_eval(model, name))

# Display
results_sorted_acc = sorted(results, key=lambda x: x[1], reverse=True)
results_sorted_time = sorted(results, key=lambda x: x[2], reverse=True)

print("=== Accuracy ranking (higher is better) ===")
for name, acc, t in results_sorted_acc:
    print(f"{name:16s}  acc={acc:.4f}  train_time={t:.3f}s")

print("\n=== Training time ranking (slower first) ===")
for name, acc, t in results_sorted_time:
    print(f"{name:16s}  train_time={t:.3f}s  acc={acc:.4f}")

best = results_sorted_acc[0][0]
slowest = results_sorted_time[0][0]
print(f"\nBest accuracy: {best}")
print(f"Slowest training: {slowest}")

# "Ең тұрақты" (жедел тексеріс): әр модельді бірнеше random_state-пен қайталап, accuracy std есептеу
def stability_test(make_model_fn, seeds=(0, 1, 2, 3, 4)):
    accs = []
    for s in seeds:
        X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=s, stratify=y)
        m = make_model_fn(s)
        m.fit(X_tr, y_tr)
        accs.append(accuracy_score(y_te, m.predict(X_te)))
    return float(np.mean(accs)), float(np.std(accs))

stab = {}
stab["Bagging"] = stability_test(lambda s: BaggingClassifier(
    estimator=DecisionTreeClassifier(random_state=s),
    n_estimators=200, random_state=s, n_jobs=-1
))
stab["RandomForest"] = stability_test(lambda s: RandomForestClassifier(
    n_estimators=500, random_state=s, n_jobs=-1
))
stab["GradientBoosting"] = stability_test(lambda s: GradientBoostingClassifier(
    n_estimators=200, random_state=s
))
stab["Stacking"] = stability_test(lambda s: StackingClassifier(
    estimators=[
        ("rf", RandomForestClassifier(n_estimators=200, random_state=s, n_jobs=-1)),
        ("svm", SVC(probability=True, random_state=s))
    ],
    final_estimator=LogisticRegression(max_iter=5000, random_state=s),
    cv=5, n_jobs=-1
))

print("\n=== Stability (mean ± std over different splits) ===")
for k, (m, sd) in stab.items():
    print(f"{k:16s}  mean={m:.4f}  std={sd:.4f}")

most_stable = min(stab.items(), key=lambda kv: kv[1][1])[0]
print(f"\nMost stable (lowest std): {most_stable}")

=== Accuracy ranking (higher is better) ===
RandomForest      acc=0.9561  train_time=1.624s
GradientBoosting  acc=0.9561  train_time=2.027s
Stacking          acc=0.9561  train_time=4.155s
Bagging           acc=0.9474  train_time=1.320s

=== Training time ranking (slower first) ===
Stacking          train_time=4.155s  acc=0.9561
GradientBoosting  train_time=2.027s  acc=0.9561
RandomForest      train_time=1.624s  acc=0.9561
Bagging           train_time=1.320s  acc=0.9474

Best accuracy: RandomForest
Slowest training: Stacking

=== Stability (mean ± std over different splits) ===
Bagging           mean=0.9667  std=0.0244
RandomForest      mean=0.9702  std=0.0163
GradientBoosting  mean=0.9649  std=0.0175
Stacking          mean=0.9667  std=0.0129

Most stable (lowest std): Stacking
